# Scraping the Cordis website

I'm not sure yet about legality of this. The website's legal information only contains information about how th users data is used, not how the user is allowed to interact with the data from the website. The only thing it mentioned is that the content is copyright protected and needs to be cited properly.

## Help Files
- [Introduction to web scraping with selenium](https://medium.com/the-andela-way/introduction-to-web-scraping-using-selenium-7ec377a8cf72)

In [44]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
#from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os

- 1st import: Allows you to launch/initialise a browser.
- 2nd import: Allows you to search for things using specific parameters.
- 3rd import: Allows you to wait for a page to load.
- 4th import: Specify what you are looking for on a specific page in order to determine that the webpage has loaded.
- 5th import: Handling a timeout situation.

## Creating helper function

create new instance of chrome in incognito mode

In [20]:
option = webdriver.ChromeOptions()
option.add_argument("— incognito")

In [21]:
chromedriver_path = "/Users/guysimons/Documents/BISS/FuturizingBusinessSchools/Scraping Python/Drivers/chromedriver"
firefoxdriver_path = '/Users/katerinadoyle/Documents/java/webscraping/drivers/geckodriver'

In [99]:
# change the xpath 
def init_driver():
    driver = webdriver.Firefox(executable_path=firefoxdriver_path) #changed this. Chrome takes too much CPU on my mac
    #driver.wait = WebDriverWait(driver, 5)
    #driver.implicitly_wait(10)
    return driver

def navigate_to_page(driver, homepage):
     driver.get(homepage)
     time.sleep(10)
     
def navigate_to_keyword_page(driver, keyword, homepage):
     driver.get(homepage)
     time.sleep(10)
     inputElement = driver.find_element_by_id("projectsResultsSearchValue")
     #select = Select(driver.find_element_by_xpath("//select[@class = 'search form-control']"))
     inputElement.send_keys(keyword)
     inputElement.send_keys(Keys.ENTER) # simulate hitting enter

def get_content(driver):
     # go to report summary page
     project_page = driver.find_element_by_xpath("//a[contains(@href,'result/rcn')]").click()
     # scrape content on result summary page
     project_info = driver.find_element_by_xpath("//div[@class = 'content']")
     return project_info
     
def next_page(driver):
     nxt_button = driver.find_element_by_xpath("//span[@data-tn-element='next-page']")
     time.sleep(2)
     nxt_button.click()
     time.sleep(2)

## Accessing the webpage and getting the data

In [63]:
homepage= 'https://cordis.europa.eu/projects/home_en.html'

In [68]:
driver = init_driver()
#time.sleep(20)

In [69]:
# go to cordis page and enter the keyword
navigate_to_keyword_page(driver, 'neurological disorder', homepage)

In [ ]:
# TODO: change default setting to see 100 entries

In [100]:
# scrape the content from page 1, go to next page, and continue
test = get_content(driver) # this works so far

In [ ]:
# TODO: content of report summary is scraped. now webdriver needs to go back, and go to next result. 


# method 1: change project_page into finding a list of urls. then loop over the urls and get the results. 

In [103]:
test.text

"ADHD drugs re-evaluated\nEuropean researchers investigated the effects of methylphenidate (MPH), a drug commonly prescribed for attention deficit hyperactivity disorder (ADHD). The outcomes of the project are useful to all groups dealing with the disorder and its effects.\n© Photographee.eu, Shutterstock\nADHD, a common childhood neurodevelopmental disorder, has high socioeconomic costs. Growing safety concerns regarding MPH led to the establishment of the EU-funded project ADDUCE (Attention deficit hyperactivity disorder drugs use chronic effects).\n\nThe consortium used multiple pharmaco-epidemiological research methods to determine the impact of MPH on children and adolescents. These included a series of retrospective and prospective pharmacovigilance studies. Also investigated were prevalence, clinical significance, development, and moderating and/or mediating factors of MPH for four specific classes: growth, neurological, psychiatric and cardiovascular.\n\nProviding a basis for s